In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [38]:
# Load data

def load_list(filename):
    vocabulary = []
    with open(filename, 'r') as f:
        for l in f:
            vocabulary.append(l.strip())
    return np.asarray(vocabulary)

def load_csv(filename):
    import csv
    
    sentence = []
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            sentence.append(str(row))
    return np.asarray(sentence).flatten()
    
pos_related = load_csv('./sentence_data/pos_related.csv')
neg_related = load_csv('./sentence_data/neg_related.csv')
pos_unrelated = load_csv('./sentence_data/pos_unrelated.csv')
neg_unrelated = load_csv('./sentence_data/neg_unrelated.csv')

In [39]:
print(pos_related.shape)
print(neg_related.shape)
print(pos_unrelated.shape)
print(neg_unrelated.shape)

(500,)
(500,)
(500,)
(500,)


In [40]:
print(pos_related[:10])

["['not perfect by a long shot, but definitely good for a smile on a bad day.']"
 "['the whole cast was great, each character had their own personality and charm.']"
 '[\'even though it has one of the standard "revenge price plot," this film is my favorite of vincent price\\\'s work.\']'
 "['i really enjoyed this movie, it is really fun to watch get elvira into all these adventure, she is just great.']"
 "['with more laugh than any other third-in-a-disney-series movie, hakuna matata is worth watching - if only for the hot tub scene which is still funny despite being a little bit predictable.']"
 "['it is really a wonderful thriller i enjoyed very much']"
 "['when my sister said this movie was gonna be good i had second thought but i watched it and it was actually funny']"
 "['it touched me in a way that, even all these year later, still affects me.']"
 "['i strongly recommend seeing for all']"
 "['without a doubt, the best late night television ever.']"]


In [41]:
print(neg_related[:10])

["['i had numerous problem with this film']"
 "['dear god i do not know where to start why this movie sucked too much']"
 "['i was pretty disappointed']"
 "['if you are tempted to watch this movie, rip your eyeball out and flush them down the toilet']"
 "['the music there was was annoying, and boring']"
 "['someone must have been seriously joking when they made this film']"
 "['ugly then, uglier now']" "['this film is predictable']"
 '[\'even the supporting male character are all "bad"\']'
 "['trust me, this is one let down movie that you want to avoid and this comes from one huge denzel washington fan']"]


In [43]:
related_set = np.hstack((pos_related, neg_related))
print(related_set.shape)
y_related = np.ones(related_set.shape)
print(y_related.shape)

# pos_set = pos_related
# y_pos = np.ones(pos_set.shape)
# neg_set = neg_related
# y_neg = np.zeros(neg_set.shape)

(1000,)
(1000,)


In [44]:
unrelated_set = np.hstack((pos_unrelated, neg_unrelated))
print(unrelated_set.shape)
y_unrelated = np.zeros(unrelated_set.shape)
print(y_unrelated.shape)

(1000,)
(1000,)


In [46]:
X = np.hstack((related_set, unrelated_set))
y = np.hstack((y_related, y_unrelated))

# X = np.hstack((pos_set, neg_set))
# y = np.hstack((y_pos, y_neg))

X.shape

(2000,)

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

token = r"(?u)\b[\w\'/]+\b"
tf_vectorizer = CountVectorizer(lowercase=True, max_df=1.0, min_df=1, binary=True, token_pattern=token)
tf_vectorizer.set_params(ngram_range=(1,1))

X_vectorized = tf_vectorizer.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.33, random_state=42)

In [48]:
words = tf_vectorizer.get_feature_names()
print(len(words))
print(words[500:600])

5770
['badly', 'badmitton', 'baffled', 'bag', 'baked', 'baker', 'bakery', 'bakshi', 'bakula', 'balance', 'balanced', 'ball', "ballentine's", 'ballet', 'bam', 'band', 'bandage', 'bang', 'banker', 'bankroll', 'banned', 'bar', 'barbara', 'bare', 'barely', 'bargain', 'barrel', 'barry', 'bart', 'based', 'basement', 'basic', 'basically', 'basis', 'basket', 'bat', 'batch', 'bathing', 'bathroom', 'battleship', 'bauer', 'bava', 'bb', 'bbc', 'bbc2', 'be', 'beach', 'bean', 'bear', 'beast', 'beat', "beatty's", 'beautiful', 'beautifully', 'became', 'because', 'beck', 'beckettian', 'become', 'becomes', 'becoming', 'bedknobs', 'bedroom', 'been', 'beer', 'before', 'began', 'begin', 'beginning', 'begins', 'behaves', 'behavior', 'behind', 'being', 'bela', 'belching', 'belief', 'believable', 'believe', 'believed', 'believer', 'bell', 'bellocchio', 'belongs', 'beloved', 'below', 'ben', 'benet', 'bent', 'berenger', 'berkley', 'beryl', 'beside', 'besides', 'best', 'bet', 'bette', 'better', 'bettered', 'betw

In [49]:
print(y_train.shape)
print(y_test.shape)

(1340,)
(660,)


In [50]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

y_predict = clf.predict(X_test)

print('Train : ', np.around(clf.score(X_train, y_train),5))
print('Test : ', np.around(clf.score(X_test, y_test),5))

Train :  0.98731
Test :  0.76818


In [51]:
print(np.sum(y_train==0))

665


In [52]:
188./(188+133)

0.5856697819314641

In [53]:
from sklearn.metrics import confusion_matrix, recall_score, classification_report

print(confusion_matrix(y_test, y_predict, labels=[1,0]))

[[246  79]
 [ 74 261]]


In [54]:
# Balanced Accuracy

np.around(recall_score(y_test, y_predict,average='weighted'), 3)

0.768

In [55]:
print(classification_report(y_test,y_predict))

             precision    recall  f1-score   support

        0.0       0.77      0.78      0.77       335
        1.0       0.77      0.76      0.76       325

avg / total       0.77      0.77      0.77       660



In [56]:
weights = clf.coef_.flatten()
def negative_positive_counts(X, y, word_index):
    neg_count = np.sum(X[y==0, word_index])
    pos_count = np.sum(X[y==1, word_index])    
    return neg_count, pos_count

In [57]:
not_related_indices = np.argsort(weights)
related_indices = not_related_indices[::-1]

print("word\tweight\trelated\tunrelated")
for i in not_related_indices[:50]:
    unrel_cnt, rel_cnt = negative_positive_counts(X_train, y_train, i)
    print("%s\t%0.2f\t%d\t%d" %(words[i], weights[i], rel_cnt, unrel_cnt))

word	weight	related	unrelated
when	-1.43	14	65
remember	-1.36	1	19
he	-1.22	13	64
day	-1.12	5	16
man	-1.11	3	21
she	-1.05	3	24
entire	-1.04	0	3
saw	-0.99	4	15
art	-0.98	0	7
they	-0.96	14	45
know	-0.90	5	16
about	-0.86	20	43
young	-0.85	1	14
into	-0.84	6	29
come	-0.82	1	5
big	-0.81	6	13
doing	-0.80	0	6
from	-0.80	30	59
between	-0.80	4	11
fear	-0.79	0	4
theater	-0.78	1	8
documantary	-0.75	1	2
we	-0.74	8	22
management	-0.74	0	2
made	-0.73	19	16
imagine	-0.72	0	4
behind	-0.71	0	6
porn	-0.70	0	4
why	-0.70	5	7
around	-0.70	2	9
event	-0.70	1	5
child	-0.69	0	8
before	-0.69	2	8
year	-0.69	10	26
comment	-0.69	0	4
look	-0.68	3	11
his	-0.68	14	47
week	-0.66	0	8
her	-0.65	6	28
brother	-0.64	0	7
intelligent	-0.64	1	3
guy	-0.64	4	13
verhoeven	-0.63	0	2
after	-0.61	12	20
reading	-0.61	2	7
house	-0.61	2	7
american	-0.61	1	7
in	-0.61	110	177
thing	-0.61	10	14
world	-0.60	1	9


In [58]:
print("word\t\tweight\trelated\tunrelated")
for i in related_indices[:50]:
    unrel_cnt, rel_cnt = negative_positive_counts(X_train, y_train, i)
    print("%s\t\t%0.2f\t%d\t%d" %(words[i], weights[i], rel_cnt, unrel_cnt))

word		weight	related	unrelated
disappointed		1.47	12	0
disappointment		1.33	7	0
recommend		1.29	20	0
boring		1.29	13	1
worth		1.27	19	0
acting		1.21	24	2
must		1.19	17	3
awful		1.18	13	1
this		1.17	273	113
character		1.14	28	9
ever		1.14	33	5
worse		1.10	8	0
watch		1.08	27	10
best		1.08	23	6
loved		1.03	7	2
very		1.02	35	7
scene		1.00	18	6
excellent		1.00	14	1
pretty		0.99	17	2
most		0.99	28	10
perfect		0.99	8	1
funny		0.97	17	4
value		0.96	7	1
otherwise		0.95	4	0
all		0.94	62	31
good		0.92	39	14
movie		0.92	175	77
recommended		0.92	6	0
film		0.91	134	61
nothing		0.90	15	4
performance		0.89	11	3
line		0.88	9	3
bad		0.87	23	8
silly		0.87	7	1
interesting		0.86	10	1
script		0.85	13	1
extremely		0.84	7	0
trash		0.82	4	0
horrible		0.82	10	1
still		0.82	15	7
waste		0.80	10	1
really		0.79	31	17
piece		0.78	8	2
done		0.76	7	1
enjoy		0.76	7	0
even		0.75	24	16
worst		0.75	18	1
great		0.75	21	9
rather		0.74	10	3
no		0.74	24	14


In [59]:
abs_indices = np.argsort(np.absolute(weights))[::-1]

print("word\tweight\trelated\tunrelated")
for i in abs_indices[:20]:
    unrel_cnt, rel_cnt = negative_positive_counts(X_train, y_train, i)
    print("%s\t%0.2f\t%d\t%d" %(words[i], weights[i], rel_cnt, unrel_cnt))

word	weight	related	unrelated
disappointed	1.47	12	0
when	-1.43	14	65
remember	-1.36	1	19
disappointment	1.33	7	0
recommend	1.29	20	0
boring	1.29	13	1
worth	1.27	19	0
he	-1.22	13	64
acting	1.21	24	2
must	1.19	17	3
awful	1.18	13	1
this	1.17	273	113
character	1.14	28	9
ever	1.14	33	5
day	-1.12	5	16
man	-1.11	3	21
worse	1.10	8	0
watch	1.08	27	10
best	1.08	23	6
she	-1.05	3	24


In [60]:
for sentence in X[:10] :
    print(sentence)

['not perfect by a long shot, but definitely good for a smile on a bad day.']
['the whole cast was great, each character had their own personality and charm.']
['even though it has one of the standard "revenge price plot," this film is my favorite of vincent price\'s work.']
['i really enjoyed this movie, it is really fun to watch get elvira into all these adventure, she is just great.']
['with more laugh than any other third-in-a-disney-series movie, hakuna matata is worth watching - if only for the hot tub scene which is still funny despite being a little bit predictable.']
['it is really a wonderful thriller i enjoyed very much']
['when my sister said this movie was gonna be good i had second thought but i watched it and it was actually funny']
['it touched me in a way that, even all these year later, still affects me.']
['i strongly recommend seeing for all']
['without a doubt, the best late night television ever.']


In [61]:
def save_pickle(path, X):
    with open(path, 'wb') as f:
        pickle.dump(X, f)
def open_pickle(path):
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_te_clean = open_pickle("./pickles/imdb_x_te_clean.pickle")
y_te = open_pickle("./pickles/imdb_y_te.pickle")

In [62]:
def print_sentence(corpus):
    from textblob import TextBlob
    text = TextBlob(corpus)
    i = 0
    sent = []
    for sentence in text.raw_sentences:
        sent.append(sentence)
    return sent

In [63]:
# import nltk
# nltk.download('punkt')
test = print_sentence(X_te_clean[0])

test_matrix = tf_vectorizer.transform(test) 
test_matrix.shape

y_pred_test = clf.predict(test_matrix)
for i in range(len(test)):
    print(y_pred_test[i], ':' , test[i])

1.0 : this was an excellent show.
0.0 : it came on pbs back home in chicago and i remember cindy herron (from envogue) played the teen aged daughter.
0.0 : the show dealt with subject such as sex, peer pressure and puberty.
0.0 : it was about a middle class black family who had a teen aged daughter and son who moved to a middle class neighborhood from oakland or somewhere (i can not remember).
0.0 : i remember several episode but the one i remember most was when their cousin got her period for the first time.
0.0 : i was probably 7-8 when i first watched it and i was able to keep up with the program.
1.0 : this was a great show.
0.0 : i can not remember the name of the guy who played the son on the show, but i always got him confused with kevin hook.


In [64]:
test = print_sentence(X_te_clean[1])

test_matrix = tf_vectorizer.transform(test) 
test_matrix.shape

y_pred_test = clf.predict(test_matrix)
x_extracted_1 = ''
print(y_te[1])
for i in range(len(test)):
    if y_pred_test[i] == 1:
        x_extracted_1 += test[i]
        print('-', test[i])

1
- age cannot tarnish the beauty of this east-west love story for me.
- as mark elliott, william holden is intelligent, breezy and a bit weak; jennifer jone is perhaps well-nigh-perfect as dr. han suyin, by turns doubt-torn and ecstatic, eager and hesitant.
- other in the large cast include torin thatcher, isobel elsom, murray matheson, virginia gregg, richard loo, soo yong, philip ahn, jorja curtright and donna martell; many of hollywood's best oriental actor played smaller uncredited part also.
- the film is unarguably physically busy, interesting and often beautiful also.
- with cinematography by leon shamroy, ben nye's makeup and helen turpin's hairstyle, the great work by set decorator, sound and lighting, art department and all concerned, this has to be one of the most memorable production set in a major non-u.s. city of all time, and one of the most difficult to capture on film.
- truly, love is a many-splendored thing, dr. han says; and this movie stands as one of that doctrin

In [65]:
test = print_sentence(X_te_clean[3])

test_matrix = tf_vectorizer.transform(test) 
test_matrix.shape

y_pred_test = clf.predict(test_matrix)
x_extracted_3 = ''
print(y_te[3])
for i in range(len(test)):
    if y_pred_test[i] == 1:
        x_extracted_3 += test[i]
        print('-',test[i])

0
- i have yet to watch the first entry in this series, however, fortunately, i was still able to follow the complex and intricate plot, with all its unexpected twist and turn, and i applaud them for the utter originality of the concept herein.
- in case there is any confusion, let me leave no doubt as to the fact that everything i have just said is coated in pure, carefully nurtured sarcasm, the kind that flourish and grow exponentially when exposed to crap like this flick.
- a clear sign that this is unimpressive is that it was directed by a visual effect creator, whose only other credit in that field is a henry rooker film that was not well received.
- the action is not terrible.
- cinematography and editing are fine.
- the music is cool enough.
- language is infrequent, if even that.
- i recommend this solely to fan of b-movie, and i will say that you can do worse than this.
- 1/10


In [66]:
## Test the clf 3

X_tr_clean = open_pickle("./pickles/imdb_x_tr_clean.pickle")
y_tr = open_pickle("./pickles/imdb_y_tr.pickle")
cv = CountVectorizer(min_df=5, token_pattern=token)
X_train_ = cv.fit_transform(X_tr_clean)

In [67]:
X_train_.shape
y_tr.shape

(25000,)

In [68]:
x_extracted = np.hstack((x_extracted_1, x_extracted_3))
X_test_ = cv.transform(x_extracted)
y_test_ = [1, 0]

In [69]:
X_test_.shape

(2, 26255)

In [70]:
clf_2 = LogisticRegression()
clf_2.fit(X_train_, y_tr)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [71]:
clf_2.score(X_test_, y_test_)

1.0